[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LLaMA-VID-jupyter/blob/main/LLaMA_VID_gradio_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaMA-VID
%cd /content/LLaMA-VID

# !pip install -q llava@git+https://github.com/haotian-liu/LLaVA.git@1619889c712e347be1cb4f78ec66e7cf414ac1a6
# !pip install -q einops fastapi gradio markdown2[all] uvicorn wandb shortuuid httpx deepspeed peft accelerate 
# !pip install -q bitsandbytes einops einops-exts timm gradio_client fairscale decord transformers==4.31.0
!pip install -q -e .
!pip install -q ninja
!pip install -q flash-attn --no-build-isolation

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://storage.googleapis.com/sfr-vision-language-research/LAVIS/models/BLIP2/eva_vit_g.pth -d /content/LLaMA-VID/model_zoo/LAVIS -o eva_vit_g.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/config.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/tokenizer.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o tokenizer.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/tokenizer_config.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/vocab.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/special_tokens_map.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/preprocessor_config.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o preprocessor_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o pytorch_model.bin

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/config.json -d /content/vid -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/generation_config.json -d /content/vid -o generation_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/pytorch_model-00001-of-00003.bin -d /content/vid -o pytorch_model-00001-of-00003.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/pytorch_model-00002-of-00003.bin -d /content/vid -o pytorch_model-00002-of-00003.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/pytorch_model-00003-of-00003.bin -d /content/vid -o pytorch_model-00003-of-00003.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/pytorch_model.bin.index.json -d /content/vid -o pytorch_model.bin.index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/special_tokens_map.json -d /content/vid -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/tokenizer.json -d /content/vid -o tokenizer.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/tokenizer.model -d /content/vid -o tokenizer.model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/tokenizer_config.json -d /content/vid -o tokenizer_config.json

In [ ]:
import threading
import subprocess
threading.Thread(target=lambda: subprocess.run(['python', '-m', 'llamavid.serve.controller', '--host', '0.0.0.0', '--port', '10000'], check=True), daemon=True).start()
# !python -m llamavid.serve.controller --host 0.0.0.0 --port 10000

In [ ]:
import threading
import subprocess
command = [
    'python', '-m', 'llamavid.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', '/content/vid',
    '--load-8bit'
]
threading.Thread(target=lambda: subprocess.run(command, check=True, shell=False), daemon=True).start()
# !python -m llamavid.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path /content/vid --load-8bit

In [ ]:
!python -m llamavid.serve.gradio_web_server --controller http://0.0.0.0:10000 --host 0.0.0.0 --model-list-mode reload --share